<a href="https://colab.research.google.com/github/uteyechea/crime-prediction-using-artificial-intelligence/blob/master/temporal_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc
import sys

import pandas as pd
from scipy import stats

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path='/content/drive/My Drive/Colab Notebooks/crime_prediction'

sys.path.append(path+'/libs')
import temporal_correlation as tc


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
prediction_file_path=os.path.join(path,'data','prediction','series_sample.txt')
prediction_file=pd.read_csv(prediction_file_path,sep=',')
print(prediction_file.shape)

(2555, 1)


In [3]:
prediction_file.iloc[:,0] = pd.to_numeric(prediction_file.iloc[:,0], errors='coerce') #Remove non float values, substitute them with NAN

In [4]:
prediction_file=prediction_file.dropna()

In [5]:
prediction_file.shape

(2527, 1)

In [6]:
prediction_file.isnull().values.any()

False

In [7]:
training_data_path=os.path.join(path,'data','training','theft2.csv')
training_file=pd.read_csv(training_data_path,sep=',')

In [8]:
training_file.isnull().values.any()

False

In [ ]:
type(prediction_file)

In [ ]:
type(training_file)

In [ ]:
prediction_file.shape

In [ ]:
training_file.shape

In [ ]:
prediction_file

In [16]:
def series_window(dataframe,end,periods,column):
  if type(column)==str:
    column=dataframe.columns.get_loc(column)
  return dataframe.iloc[end-periods:end,column]

def series_correlation(series1,series2):
  #assert something?
  series1=series1.reset_index(drop=True) #Better find a way to simplify this procedure
  series2=series2.reset_index(drop=True) #Better find a way to simplify this procedure
  ro=series1.corr(series2) 
  return ro 

def correlated_series_timestamp(dataframe1,dataframe2,end,periods,column,min_correlation):
  timestamps=[]
  series1=series_window(dataframe1,end,periods,column) #what do you want to evaluate?
  for epoch in reversed(range(2*periods,len(dataframe2))):
    series2=series_window(dataframe2,epoch-periods,periods,column)
    try:
      correlation=series_correlation(series1,series2)
      #print(correlation)
      if correlation >= min_correlation:
        #record datetime value. We will use this datetime value to generate all sequences that will go as input to the RNN
        #print(dataframe.index[epoch]) 
        timestamps.append(dataframe2.index[epoch])
    except:
      print('Can not compute correlation at index ',str(epoch))    
  return timestamps

def correlated_series_sequence(dataframe1,dataframe2,end,periods,column,min_correlation):
  timestamps=correlated_series_timestamp(dataframe1,dataframe2,end,periods,column,min_correlation)
  sequence={}
  if type(column)==int:
    column=dataframe.columns[column]
  for timestamp in timestamps:
    try:
      input[timestamp]=dataframe.loc[pd.date_range(start=timestamp,periods=periods,freq='-1D'),column]
      output[timestamp]=dataframe.loc[pd.date_range(start=timestamp,periods=periods,freq='1D',closed='right'),column]
    except:
      print('Missing date in DataFrame')
  return input,output      

def get_IO_series(dataframe,end,periods,column,min_correlation):
  timestamps=correlated_series_timestamp(dataframe,end,periods,column,min_correlation)
  input,output=correlated_series_sequence(dataframe,end,periods,column,min_correlation)
  #return input,output
  sequence={}
  for timestamp in timestamps: #Change from input to timestamp sequence
    try:
      sequence[timestamp]= input[timestamp][::-1].append(output[timestamp])
    except:
      print('Error with key',str(timestamp))
  return sequence      

def save_to_file(sequence,file_path):
  with open(file_path,'a') as file:
    for timestamp in sequence:
      sequence[timestamp].to_csv(file,mode='a',header=False,index=False)
      file.write('\n')


In [17]:
timestamps=correlated_series_timestamp(training_file,prediction_file,end=10,periods=10,column=0,min_correlation=0.85)

In [11]:
timestamps

[2365, 2347, 226]

In [18]:
timestamps

[2365, 2347, 226]